<a href="https://colab.research.google.com/github/Henry-0810/Artificial-Intelligence/blob/main/portfolio_optimization_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Genetic Algorithm
### Portfolio Optimization
I will only use 5 stocks for this project. Stock data are all real-time, retrived from the Yahoo Finance API, https://pypi.org/project/yfinance/

## **Problem Statement**

### **Description of the Problem**  
Portfolio optimization is the process of selecting the best combination of assets to maximize returns while minimizing risk. A common approach is **Markowitz’s Mean-Variance Optimization (MVO)**, which constructs an efficient portfolio using expected returns, variances, and covariances of assets.  

However, MVO has several limitations:  
1. **Complexity**: Solving quadratic optimization problems becomes expensive as the number of assets grows.  
2. **Strict Assumptions**: MVO assumes asset returns follow a normal distribution and that investors have quadratic utility functions, which is not always realistic.  
3. **Handling Constraints**: Incorporating real-world constraints (e.g., restricting allocations to certain stocks) increases the complexity.  

To overcome these challenges, this project applies a **Genetic Algorithm (GA)** to optimize portfolio allocation. Unlike MVO, GA is a heuristic approach that efficiently searches for near-optimal solutions without relying on strict mathematical assumptions.  

The goal is to maximize the **Sharpe Ratio**, a key financial metric that balances return and risk, making GA a suitable technique for solving this problem.

---

### **Discussion of the Suitability of Genetic Algorithms**  
Genetic Algorithms (GAs) are well-suited for portfolio optimization due to their ability to handle large search spaces and complex constraints. Key advantages include:  

- **Scalability** – GAs can handle large datasets efficiently without exponential growth in computational cost.  
- **No Assumption of Normality** – Unlike MVO, GA does not require asset returns to be normally distributed.  
- **Flexibility** – GA can incorporate additional constraints, such as limiting investment in specific stocks or ensuring minimum allocation to certain sectors.  

Through **evolutionary selection**, GA gradually improves the portfolio allocation, leading to **higher Sharpe Ratios** compared to naive allocation strategies.

---

### **Complexity of the Problem** (Project Plan)
Portfolio optimization presents a **high-dimensional, non-linear problem** where:  
- Each chromosome represents a possible portfolio allocation (weights og assets).  
- The solution space grows exponentially with the number of stocks considered.  
- Evaluating a portfolio requires calculating historical **returns, variances, and covariances** over a defined period.  
- The GA must balance exploration (mutation) and exploitation (crossover) to meet an optimal solution efficiently.  

The complexity of the Genetic Algorithm increases with:  
- The **number of assets** included in the portfolio (will be using only 5 in this case for a proof of concept).  
- The **historical data period** used for return calculations (longer periods increase computational load).  
- The **GA parameters** (population size, mutation rate, number of generations), which impact convergence speed.  

Despite this complexity, GA provides a robust and adaptable approach to solving portfolio optimization problems where traditional methods struggle.

---

**Reference**  
- [Markowitz Portfolio Theory](https://sites.math.washington.edu/~burke/crs/408/fin-proj/mark1.pdf)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

#### 1. Fetch Historical Data
I will fetch 6 months of historical data starting from August 11th 2024 to February 11th 2025.
I will investigate on 5 Stocks:
1. Apple Inc. (AAPL)
2. Tesla Inc. (TSLA)
3. Microsoft Corp. (MSFT)
4. Amazon.com Inc. (AMZN)
5. NVIDIA Corporation (NVDA)

In [ ]:
start_date = "2024-08-11"
end_date = "2025-02-11"

stocks = ["AAPL","TSLA","MSFT","AMZN","NVDA"]
data = yf.download(stocks, start=start_date, end=end_date, interval="1d")
print(data.columns)


[*********************100%***********************]  5 of 5 completed


MultiIndex([( 'Close', 'AAPL'),
            ( 'Close', 'AMZN'),
            ( 'Close', 'MSFT'),
            ( 'Close', 'NVDA'),
            ( 'Close', 'TSLA'),
            (  'High', 'AAPL'),
            (  'High', 'AMZN'),
            (  'High', 'MSFT'),
            (  'High', 'NVDA'),
            (  'High', 'TSLA'),
            (   'Low', 'AAPL'),
            (   'Low', 'AMZN'),
            (   'Low', 'MSFT'),
            (   'Low', 'NVDA'),
            (   'Low', 'TSLA'),
            (  'Open', 'AAPL'),
            (  'Open', 'AMZN'),
            (  'Open', 'MSFT'),
            (  'Open', 'NVDA'),
            (  'Open', 'TSLA'),
            ('Volume', 'AAPL'),
            ('Volume', 'AMZN'),
            ('Volume', 'MSFT'),
            ('Volume', 'NVDA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])


There are 5 types of Prices: Close, High, Low, Open, Volume. Close prices are often used for analysis because it reflects the final consensus on the stock price for the trading day. It is often most consistent for portfolio optimization.

In [ ]:
close_data = data["Close"]
print(close_data.head(10))

Ticker            AAPL        AMZN        MSFT        NVDA        TSLA
Date                                                                  
2024-08-12  217.052292  166.800003  404.455902  109.003159  197.490005
2024-08-13  220.784088  170.229996  411.614258  116.122063  207.830002
2024-08-14  221.233093  170.100006  414.447723  118.061768  201.380005
2024-08-15  224.226501  177.589996  419.348083  122.841026  214.139999
2024-08-16  225.553589  177.059998  416.798309  124.560760  216.119995
2024-08-19  225.393921  178.220001  419.846069  129.979919  222.720001
2024-08-20  226.012573  178.880005  423.103027  127.230354  221.100006
2024-08-21  225.902817  180.110001  422.445679  128.480164  223.270004
2024-08-22  224.036911  176.130005  413.889954  123.720886  210.660004
2024-08-23  226.341843  177.039993  415.125031  129.350021  220.320007


#### 2. Preprocess Data
